## Finetuning a Large Language Model
##### Step by step guide to performing Low Rank Adaptation For Finetuning (LoRA) of LLMs


In [ ]:
_ = !python3 -m pip install --upgrade pip
_ = !pip install accelerate -U
_ = !pip install peft
_ = !pip install datasets
_ = !pip install transformers[torch]
_ = !pip install bitsandbytes
_ = !pip install accelerate

In [ ]:
import os
import random
import string
from datetime import datetime
from typing import List
import torch
import transformers
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

### Import a LLM from HuggingFace

For more info about the used model: https://huggingface.co/Salesforce/codegen-350M-mono

In [ ]:
checkpoint = "Salesforce/codegen-350M-mono"
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

### Run an example prediction/inference

In [ ]:
text = "def hello_world():"

completion = model.generate(**tokenizer(text, return_tensors="pt").to("cuda"))

print(tokenizer.decode(completion[0]))

### Prepare model for training

In [ ]:
GRADIENT_CHECKPOINTING = False
PER_DEVICE_TRAIN_BATCH_SIZE = 1
WARMUP_STEPS = 0
EPOCHS = 100
LEARNING_RATE = 1e-5
R = 32
LORA_ALPHA = R
LORA_DROPOUT = 0.1

In [ ]:
def find_target_modules(model) -> List[str]:
    """
    Identify linear layers in the model and return as list.
    """
    layers = set()
    for name, module in model.named_modules():
        if "Linear" in str(type(module)):
            layer_type = name.split('.')[-1]
            layers.add(layer_type)

    return list(layers)

target_modules = find_target_modules(model)
target_modules

In [ ]:
peft_config = LoraConfig(
    task_type="Causal_LM", inference_mode=False, r=R, lora_alpha=LORA_ALPHA, lora_dropout=0.1, target_modules=target_modules
)

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
letters = string.ascii_letters

prompts = []
length = 100
for i in range(3):
    random_string = ''.join(random.choice(letters) for i in range(length))
    prompts.append("def hello_world():" + random_string)

data = [{"text": x} for x in prompts]
dataset = Dataset.from_dict({"text": [item["text"] for item in data[:]]})
tokenized_dataset = dataset.map(lambda x : tokenizer(x["text"]), batched=True)

In [ ]:
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    warmup_steps=WARMUP_STEPS,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    logging_steps=100,
    save_total_limit=1,
    output_dir=os.path.join('.', datetime.now().strftime("%Y%m%d%H%M%S")),
    gradient_checkpointing=GRADIENT_CHECKPOINTING
)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=train_args,
    callbacks=[],
    data_collator=transformers.DataCollatorForLanguageModeling(
        tokenizer, mlm=False
    )
)

### Start training

In [ ]:
loss = trainer.train()

### Run an inference after fine-tuning

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
text = "def hello_world():"

completion = model.generate(**tokenizer(text, return_tensors="pt").to("cuda"), max_length=100)

print(tokenizer.decode(completion[0]))